# LMD - Simple Autoencoder

Demonstrate how an autoencoder remembers input images pixel by pixel.

1. Define dimensions of the image and resize input images accordingly
2. Train autoencoder. Throughout each step, save a copy of the reconstruction of the image
3. Display reconstructed images

In [ ]:
# Import necessary libraries and path relative to project
import torch
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import pandas as pd

import sys
import os
import cv2
import glob
import numpy as np
import math
from matplotlib import pyplot as plt

%matplotlib inline

sys.path.append(os.path.join(os.path.abspath(''), '../pyno/lib'))

from autoencoder import Autoencoder
from abstract_dataset import AbstractDataset

if not torch.cuda.is_available():
    raise Exception("CUDA not available")
    
# CUDA related information
print('__Python VERSION:', sys.version)
print('__pyTorch VERSION:', torch.__version__)
print('__CUDA VERSION', )
from subprocess import call
# call(["nvcc", "--version"]) does not work
! nvcc --version
print('__CUDNN VERSION:', torch.backends.cudnn.version())
print('__Number CUDA Devices:', torch.cuda.device_count())
print('__Devices')
# call(["nvidia-smi", "--format=csv", "--query-gpu=index,name,driver_version,memory.total,memory.used,memory.free"])
print('Active CUDA Device: GPU', torch.cuda.current_device())
print ('Available devices ', torch.cuda.device_count())
print ('Current cuda device ', torch.cuda.current_device())

torch.cuda.empty_cache()
import gc
gc.collect()

## Parameters

In [ ]:
# Image
image_height = 800
image_width = 800

dir_input_images = "/home/ralampay/Pictures/niece"
dir_output_images = "/home/ralampay/Pictures/niece-lmd"

# Device for training

# The topology of the model from input layer to innermost latent layer
input_dim = image_height * image_width
hidden_layer_dimension = int(10)

layers = [input_dim, hidden_layer_dimension]
print(layers)

device = 'cuda:0'
h_activation = 'relu'
o_activation = 'sigmoid'
device = torch.device(device)
error_type = 'mse'
optimizer_type = 'adam'

In [ ]:
# Initialize the autoencoder
autoencoder = Autoencoder(
                layers=layers, 
                h_activation=h_activation, 
                o_activation=o_activation, 
                device=device, 
                error_type=error_type, 
                optimizer_type=optimizer_type)

## Build the Dataset as a vector of pixels

In [ ]:
ext = ['png', 'jpg', 'gif']    # Add image formats here

files = []
[files.extend(glob.glob(dir_input_images + '/*.' + e)) for e in ext]

dim = (image_width, image_height)

images = np.array([cv2.resize(cv2.imread(file, cv2.IMREAD_GRAYSCALE), dim) for file in files])

for image in images:
    plt.imshow(image)
    plt.show()
    
input_data = np.array([image.ravel() / 255 for image in images])
print(input_data)
print(input_data.shape)

## Setup Tensor Data

In [ ]:
x = torch.tensor(input_data).float().to(device)

## Training

In [ ]:
epochs = 100
lr = 0.01
batch_size = 5

data = AbstractDataset(x)
dataloader = DataLoader(dataset=data, batch_size=batch_size, shuffle=True, drop_last=False)

autoencoder.optimizer = optim.Adam(autoencoder.parameters(), lr=lr)

num_iterations = data.n_samples / batch_size

counter = 0

for epoch in range(epochs):
    curr_loss = 0
    
    for i, (inputs, labels) in enumerate(dataloader):
        inputs, labels = inputs.to(device), labels.to(device)
        autoencoder.optimizer.zero_grad()
        
        output = autoencoder.forward(inputs)
        
        
        loss = (output - labels).pow(2).sum(dim=1).sqrt().mean()
        
        curr_loss += loss
        
        loss.backward()
        
        autoencoder.optimizer.step()
        
        # display the images
        raw_output = output.detach().cpu().numpy()
        
        for o in raw_output:
            output_image = np.reshape(o * 255, (image_width, image_height))
            file_to_write = "{}/{}.jpg".format(dir_output_images, str(counter))
            print("File to write: {}".format(file_to_write))
            cv2.imwrite(file_to_write, output_image)
            plt.imshow(output_image)
            plt.show()
            
            counter = counter + 1
            
    curr_loss = curr_loss / num_iterations